In [44]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import joblib
import os
import sys
from collections import defaultdict
from Codes.import_data import reformat_cgc
import csv
import tqdm
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import joblib
from sklearn.model_selection import train_test_split

In [3]:
input_file = 'cgc_standard.out'
output_file = 'reformat'
reformat_cgc(input_file, output_file)

In [4]:
# Define input and output file paths
input_file_path = output_file  # 
output_file_path = 'output.csv'  

# Open the input file for reading and the output file for writing
with open(input_file_path, 'r') as infile, open(output_file_path, 'w', newline='') as outfile:
    reader = infile.readlines()
    writer = csv.writer(outfile)
    
    # Write the header to the CSV file
    writer.writerow(['cgc_id', 'sequence'])
    
    # Process each line from the input file
    for line in reader:
        parts = line.strip().split('\t')  # Split the line into two parts
        writer.writerow(parts)  # Write the parts to the CSV file
#remove files        
os.remove('reformat')

In [5]:
def custom_tokenizer(x):
    return str(x).replace("|", ",").split(',')
model = joblib.load('best_model.joblib')
le = joblib.load('label_encoder.pkl')

In [6]:
############################################
#Import csv file 
############################################

data = pd.read_csv(output_file_path)
test_seqs = np.array([test_item.replace("|", ",").replace(",", " ") for test_item in data["sequence"].values])
label = data['cgc_id']
os.remove(output_file_path)

In [7]:
data_train = pd.read_csv('Data/Train_data.csv')

In [ ]:
# Define parameters directly
n_estimators = 100
class_weight = "balanced"
sampling_strategy = "all"
replacement = True
bootstrap = False

# Get order of classes
order = list(data_train["high_level_substr"].value_counts().index)

# Create the classifier with direct parameter assignment
balanced_rf = BalancedRandomForestClassifier(
    n_estimators=n_estimators,
    class_weight=class_weight,
    sampling_strategy=sampling_strategy,
    replacement=replacement,
    bootstrap=bootstrap,
    n_jobs=-1
)

# Create and train pipeline
clf_one_vs_rest = Pipeline([
    ('vectorizer', CountVectorizer(
        tokenizer=custom_tokenizer, 
        lowercase=False
    )), 
    ('vr', OneVsRestClassifier(balanced_rf))
])

# Fit the model directly
clf_one_vs_rest.fit(data_train['sig_gene_seq'].values, data_train["high_level_substr"].values)

/home/yifan/miniconda3/envs/subfinder/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vectorizer',
                 CountVectorizer(lowercase=False,
                                 tokenizer=<function custom_tokenizer at 0x7f88cc9b9c60>)),
                ('vr',
                 OneVsRestClassifier(estimator=BalancedRandomForestClassifier(bootstrap=False,
                                                                              class_weight='balanced',
                                                                              n_jobs=-1,
                                                                              replacement=True,
                                                                              sampling_strategy='all')))])

In [48]:
## To get easy to understand p values
## we need to change the model a bit

class_order = clf_one_vs_rest.classes_

# get the predictions
preds_df = pd.DataFrame()
counter = 0
outer_catch = []


# we will store the normalized probabilities here
normalized_probs = np.zeros((len(test_seqs), len(class_order)))


In [49]:
import pandas as pd
import gensim
import numpy as np
from tqdm import tqdm
from imblearn.ensemble import BalancedRandomForestClassifier
import matplotlib.pyplot as plt
from scipy.stats import binom
from sklearn.multiclass import OneVsRestClassifier

In [50]:
how_many = 5
for inner in tqdm(range(0, how_many)):
    balanced_rf = BalancedRandomForestClassifier(n_jobs=7, class_weight="balanced", random_state=inner)
    
    model = Pipeline([
        ('vectorizer', CountVectorizer(tokenizer=custom_tokenizer, lowercase=False)), 
        ('vr', OneVsRestClassifier(balanced_rf))
    ])
    
    X_train, X_val, y_train, y_val = train_test_split(
        data_train['sig_gene_seq'].values, 
        data_train["high_level_substr"].values, 
        test_size=0.2, random_state=inner
    )
    
    model.fit(X_train, y_train)
    
    preds_proba = model.predict_proba(test_seqs)
    normalized_probs += preds_proba  
    ## grab all the separate one vs all estimators
    ova_ests = model.named_steps['vr'].estimators_
    inner_counter = 0
    inner_catch = []
    
    for ests in ova_ests:
        # We need to use the vectorizer first
        vectorized_test_seqs = model.named_steps['vectorizer'].transform(test_seqs)
        # Now predict on the transformed data
        preds = ests.predict_proba(vectorized_test_seqs)
        preds_df1 = pd.DataFrame(preds)
        preds_df1 = preds_df1[[1]]
        preds_df1 = pd.DataFrame(preds_df1)
        preds_df1.columns = ["unnormalized_prob_" + class_order[inner_counter] + "_" + str(counter)]
        inner_catch.append(preds_df1)
        inner_counter += 1
    
    inner_catch_df = pd.concat(inner_catch, axis=1)
    outer_catch.append(inner_catch_df)
    counter += 1

  0%|          | 0/5 [00:00<?, ?it/s]/home/yifan/miniconda3/envs/subfinder/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/yifan/miniconda3/envs/subfinder/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
/home/yifan/miniconda3/envs/subfinder/lib/python3.10/site-packages/imblearn/ensemble/_forest.py:589: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(
/home/yifan/miniconda3/envs/subfinder/l

In [38]:
## ensemble of 15 models
how_many = 5

for inner in tqdm(range(0, how_many)):
    ## one vs all model
    model = Pipeline([
        ('vectorizer', CountVectorizer(
            tokenizer=custom_tokenizer, 
            lowercase=False
        )), 
        ('vr', OneVsRestClassifier(balanced_rf))
    ])
    
    # The error is happening because test_seqs contains string data
    # that hasn't been vectorized yet
    model.fit(data_train['sig_gene_seq'].values, data_train["high_level_substr"].values)
    
    # These lines are correct - the pipeline handles vectorization
    preds_full = model.predict(test_seqs)
    preds_proba = model.predict_proba(test_seqs)
    normalized_probs += preds_proba
    
    ## grab all the separate one vs all estimators
    ova_ests = model.named_steps['vr'].estimators_
    inner_counter = 0
    inner_catch = []
    
    for ests in ova_ests:
        # We need to use the vectorizer first
        vectorized_test_seqs = model.named_steps['vectorizer'].transform(test_seqs)
        # Now predict on the transformed data
        preds = ests.predict_proba(vectorized_test_seqs)
        preds_df1 = pd.DataFrame(preds)
        preds_df1 = preds_df1[[1]]
        preds_df1 = pd.DataFrame(preds_df1)
        preds_df1.columns = ["unnormalized_prob_" + class_order[inner_counter] + "_" + str(counter)]
        inner_catch.append(preds_df1)
        inner_counter += 1
    
    inner_catch_df = pd.concat(inner_catch, axis=1)
    outer_catch.append(inner_catch_df)
    counter += 1


  0%|          | 0/5 [00:00<?, ?it/s]

/home/yifan/miniconda3/envs/subfinder/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
 20%|██        | 1/5 [00:13<00:52, 13.18s/it]/home/yifan/miniconda3/envs/subfinder/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
 40%|████      | 2/5 [00:26<00:39, 13.19s/it]/home/yifan/miniconda3/envs/subfinder/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
 60%|██████    | 3/5 [00:39<00:26, 13.14s/it]/home/yifan/miniconda3/envs/subfinder/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
 80%|

In [51]:
# Combine stored probability data into a single DataFrame
outer_catch_df = pd.concat(outer_catch, axis=1)

# Ensure column names are sorted
outer_catch_df = outer_catch_df[np.sort(outer_catch_df.columns)]

# Assign sequences to DataFrame
outer_catch_df["sequence"] = test_seqs

# Reorder columns to move 'sequence' to the front
cols = ["sequence"] + [col for col in outer_catch_df.columns if col != "sequence"]
outer_catch_df = outer_catch_df[cols]

# Normalize probabilities
normalized_probs /= how_many
normalized_probs = pd.DataFrame(normalized_probs, columns=["normalized_prob_" + item for item in class_order])

# Assign sequence identifiers
normalized_probs["sequence"] = test_seqs

# Reorder columns to move 'sequence' to the front
cols = ["sequence"] + [col for col in normalized_probs.columns if col != "sequence"]
normalized_probs = normalized_probs[cols]

# Determine the most probable class
normalized_probs["predicted_substrate"] = normalized_probs.iloc[:, 1:].idxmax(axis=1)
normalized_probs["predicted_substrate"] = normalized_probs["predicted_substrate"].apply(lambda x: x.split("_")[-1])

# Reshape data for further analysis
outer_catch_df = outer_catch_df.melt(id_vars=["sequence"]).reset_index(drop=True)

# Extract relevant information from variable names
outer_catch_df["substrate"] = outer_catch_df["variable"].apply(lambda x: x.split("_")[-2])
outer_catch_df["order"] = outer_catch_df["variable"].apply(lambda x: x.split("_")[-1])

# Sort values for consistency
outer_catch_df = outer_catch_df.sort_values(["sequence", "substrate"], ascending=[True, True])

# Classify probabilities into binary predictions
outer_catch_df["yes_or_no"] = (outer_catch_df["value"] > 0.5).astype(int)

# Aggregate probability and binary classification results
outer_catch_df_summary = (
    outer_catch_df
    .groupby(["sequence", "substrate"])
    .agg(probability_score=("value", "mean"), successes=("yes_or_no", "sum"))
    .reset_index()
)

# Define statistical function for p-value computation
def p_value_function(successes, trials=how_many, prob=0.5):
    return 1 - binom.cdf(successes, trials, prob)

# Compute p-values and clean up DataFrame
outer_catch_df_summary["p_value"] = outer_catch_df_summary["successes"].apply(p_value_function)
outer_catch_df_summary = outer_catch_df_summary.drop(columns=["successes"])

# Final sorting and resetting index
outer_catch_df_summary = outer_catch_df_summary.sort_values(["sequence", "p_value"]).reset_index(drop=True)


In [52]:
outer_catch_df_summary

,sequence,substrate,probability_score,p_value
0,1.A.33 GT2 null null 2.A.19 Aminotran_1_2 null...,alpha-glucan,0.770000,0.00000
1,1.A.33 GT2 null null 2.A.19 Aminotran_1_2 null...,beta-glucan,0.789563,0.00000
2,1.A.33 GT2 null null 2.A.19 Aminotran_1_2 null...,beta-mannan,0.692000,0.00000
3,1.A.33 GT2 null null 2.A.19 Aminotran_1_2 null...,galactan,0.716000,0.00000
4,1.A.33 GT2 null null 2.A.19 Aminotran_1_2 null...,host glycan,0.600120,0.00000
...,...,...,...,...
283,zf-C3HC4 null 3.A.8 Pribosyltran null GH47,xylan,0.498000,0.18750
284,zf-C3HC4 null 3.A.8 Pribosyltran null GH47,arabinogalactan,0.434000,0.81250
285,zf-C3HC4 null 3.A.8 Pribosyltran null GH47,alginate,0.236000,0.96875
286,zf-C3HC4 null 3.A.8 Pribosyltran null GH47,alpha-mannan,0.247966,0.96875


In [53]:


outer_catch_df_summary.to_csv("Predictions_rumen_with_probability_and_p_values_Blast_Style_new_sup_new_unsup.csv", index = False)
normalized_probs["predicted_substrate"].value_counts()

predicted_substrate
beta-glucan    24
Name: count, dtype: int64